In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import requests
import tensorflow as tf
import time
from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
def fetchData():
    res = requests.post("http://localhost:7001/api/etl/shdr")
    result = res.json()
    return result
  

In [5]:
df = fetchData()
df = pd.json_normalize(df)
df

ConnectionError: HTTPConnectionPool(host='localhost', port=7001): Max retries exceeded with url: /api/etl/shdr (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x16ace5040>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
dataset = df[['flows', 'isMon', 'isFri', 'isSaturday', 'isSunday', 'isHoliday', 'price', 'monthLevel', 'weaLevel', 'ANYDAY', 'FLEXPASS_SUNDAY', 'SUNDAY', 'WEEKDAY_SUNDAY']]
dataset = dataset[1:]
dataset


: 

In [ ]:
# 拆分训练数据集和测试数据集
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
dataset.drop(train_dataset.index)

: 

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop("flows")
train_stats = train_stats.transpose()

train_stats

: 

In [ ]:
# 从标签中分离特征
train_labels = train_dataset.pop('flows')
test_labels = test_dataset.pop('flows')

normed_train_data = train_dataset
normed_test_data = test_dataset

: 

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

: 

In [ ]:
model = build_model()
model.summary()

example_batch = normed_train_data[:3]
example_result = model.predict(example_batch)
example_result

: 

In [ ]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 500

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

: 

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

: 

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [Total]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,1])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$Total^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,1])
  plt.legend()
  plt.show()


plot_history(history)

: 

In [ ]:
model = build_model()

# patience 值用来检查改进 epochs 的数量
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS, validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

: 

In [ ]:
# 做预测
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [Total]')
plt.ylabel('Predictions [Total]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100000, 100000], [-100000, 100000])

: 

In [ ]:
model.save('../model/shdr_flows.h5')

normed_test_data

: 

: 